**PART B — ADAPTATION TO INDIAN CONTEXT (UTTARAKHAND - HIMALAYAS)**

This notebook will cover:

1. Context & SDG relevance

2. LandCover.ai training dataset (proxy labels)

3. Improved Attention U-Net architecture

4. Training on proxy labels

5. Applying model to Sentinel-2 imagery (Joshimath)

6. NDVI-guided inference

7. Yearly change detection maps (2016-2023)


# NOTE ON TRAINING RUNTIME / GPU LIMITS

The experimental design for this TensorFlow Attention U-Net was to train
for 25 epochs on all available LandCover.ai tiles (batch_size=4).

On Google Colab, the complete 25-epoch run could not always be finished
due to GPU runtime limits and session resets. However, the *identical*
architecture, tiling strategy, and loss function were successfully trained
end-to-end in the PyTorch implementation (myaisd_cw2_part2.py), where a
12-epoch run converged and achieved stable validation mIoU.

Therefore, AISD_CW2_partB.py documents the full TensorFlow pipeline and the
intended experiment, while myaisd_cw2_part2.py provides the fully completed
training run used for quantitative analysis in the report.

In [ ]:
import os

DATA_ROOT = "/content/LandCoverAI"
RAW_DIR = "/content/LandCoverAI/raw"
TILES_IMG_DIR = "/content/LandCoverAI/tiles/images"
TILES_MASK_DIR = "/content/LandCoverAI/tiles/masks"

os.makedirs(DATA_ROOT, exist_ok=True)
os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(TILES_IMG_DIR, exist_ok=True)
os.makedirs(TILES_MASK_DIR, exist_ok=True)

DATA_ROOT

'/content/LandCoverAI'

1. Context & SDG Motivation (India - Uttarakhand Himalayas)

The Himalayan state of Uttarakhand has undergone extensive land-use change and forest loss, particularly around Joshimath, where deforestation-driven slope instability contributed directly to the 2023 land subsidence crisis.
This region is relevant to multiple SDGs:

SDG 13 - Climate Action

SDG 15 - Life on Land

SDG 11 - Sustainable Cities & Communities

The goal of Part B is to adapt the Attention U-Net model from the Amazon rainforest dataset to the Indian Himalayan context, where official high-resolution annotated datasets are unavailable.
Therefore, I use LandCover.ai as a proxy training dataset and apply the trained model to Sentinel-2 imagery of Joshimath to detect vegetation loss over time.

In [ ]:
!wget -O /content/LandCoverAI/landcoverai.zip \
https://landcover.ai.linuxpolska.com/download/landcover.ai.v1.zip

--2025-12-10 00:25:37--  https://landcover.ai.linuxpolska.com/download/landcover.ai.v1.zip
Resolving landcover.ai.linuxpolska.com (landcover.ai.linuxpolska.com)... 195.78.67.65
Connecting to landcover.ai.linuxpolska.com (landcover.ai.linuxpolska.com)|195.78.67.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1538212277 (1.4G) [application/zip]
Saving to: ‘/content/LandCoverAI/landcoverai.zip’

/content/LandCoverA 100%[===================>]   1.43G  10.3MB/s    in 2m 22s  

2025-12-10 00:28:00 (10.3 MB/s) - ‘/content/LandCoverAI/landcoverai.zip’ saved [1538212277/1538212277]



In [ ]:
import zipfile

zip_path = "/content/LandCoverAI/landcoverai.zip"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(RAW_DIR)

print("Extracted into:", RAW_DIR)

Extracted into: /content/LandCoverAI/raw


LandCover.ai originally contains 41 BIG orthophotos -

Each orthophoto is huge: Up to 9000 × 9500 px, RGB GeoTIFF and paired segmentation mask

In [ ]:
import cv2
import glob

RAW_IMG_DIR = "/content/LandCoverAI/raw/images/"
RAW_MASK_DIR = "/content/LandCoverAI/raw/masks/"

img_paths = sorted(glob.glob(RAW_IMG_DIR + "*.tif"))
mask_paths = sorted(glob.glob(RAW_MASK_DIR + "*.tif"))

TILE_SIZE = 512

for img_path, mask_path in zip(img_paths, mask_paths):
    img_name = os.path.splitext(os.path.basename(img_path))[0]

    img = cv2.imread(img_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    h, w = img.shape[:2]
    k = 0

    for y in range(0, h, TILE_SIZE):
        for x in range(0, w, TILE_SIZE):
            tile_img = img[y:y+TILE_SIZE, x:x+TILE_SIZE]
            tile_mask = mask[y:y+TILE_SIZE, x:x+TILE_SIZE]

            if tile_img.shape[0] == TILE_SIZE and tile_img.shape[1] == TILE_SIZE:
                cv2.imwrite(f"{TILES_IMG_DIR}/{img_name}_{k}.jpg", tile_img)
                cv2.imwrite(f"{TILES_MASK_DIR}/{img_name}_{k}_m.png", tile_mask)
            k += 1

    print(f"Processed {img_name}")

Processed M-33-20-D-c-4-2
Processed M-33-20-D-d-3-3
Processed M-33-32-B-b-4-4
Processed M-33-48-A-c-4-4
Processed M-33-7-A-d-2-3
Processed M-33-7-A-d-3-2
Processed M-34-32-B-a-4-3
Processed M-34-32-B-b-1-3
Processed M-34-5-D-d-4-2
Processed M-34-51-C-b-2-1
Processed M-34-51-C-d-4-1
Processed M-34-55-B-b-4-1
Processed M-34-56-A-b-1-4
Processed M-34-6-A-d-2-2
Processed M-34-65-D-a-4-4
Processed M-34-65-D-c-4-2
Processed M-34-65-D-d-4-1
Processed M-34-68-B-a-1-3
Processed M-34-77-B-c-2-3
Processed N-33-104-A-c-1-1
Processed N-33-119-C-c-3-3
Processed N-33-130-A-d-3-3
Processed N-33-130-A-d-4-4
Processed N-33-139-C-d-2-2
Processed N-33-139-C-d-2-4
Processed N-33-139-D-c-1-3
Processed N-33-60-D-c-4-2
Processed N-33-60-D-d-1-2
Processed N-33-96-D-d-1-1
Processed N-34-106-A-b-3-4
Processed N-34-106-A-c-1-3
Processed N-34-140-A-b-3-2
Processed N-34-140-A-b-4-2
Processed N-34-140-A-d-3-4
Processed N-34-140-A-d-4-2
Processed N-34-61-B-a-1-1
Processed N-34-66-C-c-4-3
Processed N-34-77-A-b-1-4
Pro

In [ ]:
print("Tiled Images:", len(os.listdir(TILES_IMG_DIR)))
print("Tiled Masks:", len(os.listdir(TILES_MASK_DIR)))

Tiled Images: 10674
Tiled Masks: 10674


In [ ]:
# session ram crashed so changing to a Lightweight Data Generator, will load only what is needed per batch.
import glob

TILES_IMG_DIR = "/content/LandCoverAI/tiles/images/"
TILES_MASK_DIR = "/content/LandCoverAI/tiles/masks/"

image_paths = sorted(glob.glob(TILES_IMG_DIR + "*.jpg"))
mask_paths = sorted(glob.glob(TILES_MASK_DIR + "*_m.png"))

print("Images:", len(image_paths))
print("Masks:", len(mask_paths))


Images: 10674
Masks: 10674


In [ ]:
from sklearn.model_selection import train_test_split

train_img, val_img, train_mask, val_mask = train_test_split(
    image_paths, mask_paths, test_size=0.2, random_state=42
)

In [ ]:
#augmentation and data generator that loads files on demand
import numpy as np
import cv2
import tensorflow as tf

def tile_generator(img_paths, mask_paths, batch_size=4):
    while True:
        idx = np.random.choice(len(img_paths), batch_size)

        batch_imgs = []
        batch_masks = []

        for i in idx:
            img = cv2.imread(img_paths[i], cv2.IMREAD_COLOR)
            mask = cv2.imread(mask_paths[i], cv2.IMREAD_GRAYSCALE)

            mask = (mask > 0).astype("float32")

            batch_imgs.append(img)
            batch_masks.append(mask)

        batch_imgs = np.array(batch_imgs)
        batch_masks = np.array(batch_masks).reshape(-1, 512, 512, 1)

        yield batch_imgs, batch_masks

In [ ]:
batch_size = 4

train_gen = tile_generator(train_img, train_mask, batch_size=batch_size)
val_gen   = tile_generator(val_img, val_mask, batch_size=batch_size)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

# Attention Gate
def attention_block(x, g, inter_channels):
    theta_x = layers.Conv2D(inter_channels, (2, 2), strides=(2, 2), padding='same')(x)
    phi_g = layers.Conv2D(inter_channels, (1, 1), padding='same')(g)

    add_xg = layers.Activation('relu')(layers.add([theta_x, phi_g]))
    psi = layers.Conv2D(1, (1, 1), padding='same')(add_xg)
    psi = layers.Activation('sigmoid')(psi)
    psi_up = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(psi)

    return layers.multiply([x, psi_up])

# Attention U-Net
def attention_unet(input_shape=(512, 512, 3)):
    inputs = layers.Input(input_shape)

    # Encoder
    c1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, 3, activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, 3, activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, 3, activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, 3, activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, 3, activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, 3, activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, 3, activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    # Bottleneck
    bn = layers.Conv2D(1024, 3, activation='relu', padding='same')(p4)
    bn = layers.Conv2D(1024, 3, activation='relu', padding='same')(bn)

    # Decoder + Attention
    g1 = layers.Conv2D(512, 1)(bn)
    att1 = attention_block(c4, g1, 256)
    u1 = layers.UpSampling2D((2, 2))(bn)
    u1 = layers.concatenate([u1, att1])
    c5 = layers.Conv2D(512, 3, activation='relu', padding='same')(u1)
    c5 = layers.Conv2D(512, 3, activation='relu', padding='same')(c5)

    g2 = layers.Conv2D(256, 1)(c5)
    att2 = attention_block(c3, g2, 128)
    u2 = layers.UpSampling2D((2, 2))(c5)
    u2 = layers.concatenate([u2, att2])
    c6 = layers.Conv2D(256, 3, activation='relu', padding='same')(u2)
    c6 = layers.Conv2D(256, 3, activation='relu', padding='same')(c6)

    g3 = layers.Conv2D(128, 1)(c6)
    att3 = attention_block(c2, g3, 64)
    u3 = layers.UpSampling2D((2, 2))(c6)
    u3 = layers.concatenate([u3, att3])
    c7 = layers.Conv2D(128, 3, activation='relu', padding='same')(u3)
    c7 = layers.Conv2D(128, 3, activation='relu', padding='same')(c7)

    g4 = layers.Conv2D(64, 1)(c7)
    att4 = attention_block(c1, g4, 32)
    u4 = layers.UpSampling2D((2, 2))(c7)
    u4 = layers.concatenate([u4, att4])
    c8 = layers.Conv2D(64, 3, activation='relu', padding='same')(u4)
    c8 = layers.Conv2D(64, 3, activation='relu', padding='same')(c8)

    outputs = layers.Conv2D(1, (1, 1), padding='same', activation='sigmoid')(c8)

    return Model(inputs, outputs)

In [ ]:
model = attention_unet()

In [ ]:
# 1. Tversky Index
def tversky(y_true, y_pred, alpha=0.7):
    y_true_pos = tf.reshape(y_true, [-1])
    y_pred_pos = tf.reshape(y_pred, [-1])
    true_pos = tf.reduce_sum(y_true_pos * y_pred_pos)
    false_neg = tf.reduce_sum(y_true_pos * (1 - y_pred_pos))
    false_pos = tf.reduce_sum((1 - y_true_pos) * y_pred_pos)
    return (true_pos + 1e-6) / (
        true_pos + alpha * false_neg + (1 - alpha) * false_pos + 1e-6
    )

# 2. Focal Tversky Loss
def focal_tversky_loss(y_true, y_pred):
    return tf.pow((1 - tversky(y_true, y_pred)), 1.3)

# 3. Combo Loss (Focal Tversky + BCE)
def combo_loss(y_true, y_pred):
    ft = focal_tversky_loss(y_true, y_pred)
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    return 0.7 * ft + 0.3 * bce

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=combo_loss,
    metrics=["accuracy"]
)

In [ ]:
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=4,
    min_lr=1e-6,
    verbose=1
)

In [ ]:
steps = len(train_img) // batch_size
val_steps = len(val_img) // batch_size

history = model.fit(
    train_gen,
    steps_per_epoch=steps,
    validation_data=val_gen,
    validation_steps=val_steps,
    epochs=25,
    callbacks=[lr_callback],
    verbose=1
)

Epoch 1/25
 603/2134 ━━━━━━━━━━━━━━━━━━━━ 24:55 977ms/step - accuracy: 0.4249 - loss: 0.5396